<a href="https://colab.research.google.com/github/lisa11323/CSR_Laddit/blob/Study-1/study1_zero_shot_%EC%88%98%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Preprocessed Data

In [ ]:
!pip install -U pandas numpy
!pip install nltk emoji

In [ ]:
import pandas as pd
import re
import emoji
from tqdm import tqdm
from google.colab import files

# 📁 파일 업로드
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# ⏳ tqdm과 pandas 연동
tqdm.pandas()

# 📄 엑셀 파일 읽기
df = pd.read_excel(file_name)

# 🧹 이모지 + 비영어 문자 제거 함수
def clean_text(text):
    if isinstance(text, float):
        text = str(text)
    text = emoji.replace_emoji(text, replace="")
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text.strip()

# 🔄 전처리 적용 (기존 'Review' 덮어쓰기)
df['Review'] = df['Review'].progress_apply(clean_text)

# ✨ 리뷰 길이 계산 함수
def get_review_length(text):
    try:
        return len(str(text).split())
    except:
        return 0

# 📏 Review_length 컬럼 생성
df['Review_length'] = df['Review'].progress_apply(get_review_length)

# 🚫 리뷰 길이 10 미만 제거, 기준을 바꾸려면 여기에 있는 숫자 10을 수정
df = df[df['Review_length'] >= 10].reset_index(drop=True)

# ✅ 엑셀로 저장
preprocessed_file_path = 'preprocessed_comments_with_length.xlsx'
df.to_excel(preprocessed_file_path, index=False)
print(f"✅ 전처리 + 리뷰 길이 ≥10 필터링 데이터를 '{preprocessed_file_path}' 파일로 저장했습니다.")

# 📥 다운로드
files.download(preprocessed_file_path)

## 제로샷 기반 사회적 자본 구성요소 측정 코드

1. trust

In [ ]:
# 📚 라이브러리 불러오기
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from google.colab import files

# ⬆️ 엑셀 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 엑셀 데이터 불러오기
df = pd.read_excel(filename)

# ✅ 'comment' 열 존재 확인
if 'comment' not in df.columns:
    raise ValueError("❌ 'comment' 열이 엑셀에 없습니다!")

# 🤖 Zero-shot 분류기 로드
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 🎯 Trust 라벨 정의 (5단계)
trust_labels = [
    ("Very Unrelated", 0.0),
    ("Somewhat Unrelated", 0.25),
    ("Neutral", 0.5),
    ("Somewhat Related", 0.75),
    ("Very Related", 1.0)
]

# 📝 Hypothesis 문장 정의
hypothesis_template = "This comment is {} in terms of the trust component of social capital."

# 🧠 Trust 점수 계산 함수
def classify_trust_score(comment):
    if pd.isna(comment) or not str(comment).strip():
        return 0.0

    labels = [label for label, _ in trust_labels]

    output = classifier(
        comment,
        candidate_labels=labels,
        hypothesis_template=hypothesis_template,
        return_all_scores=True
    )

    # output 검증 및 가중 평균 계산
    if isinstance(output, list):
        result = output
    elif isinstance(output, dict) and "scores" in output:
        result = [
            {'label': label, 'score': score}
            for label, score in zip(output["labels"], output["scores"])
        ]
    else:
        raise ValueError("모델 출력 형식이 예상과 다릅니다")

    weighted = sum(r['score'] * w for r, (_, w) in zip(result, trust_labels))
    return round(weighted, 4)

# 🏃 분석 실행
tqdm.pandas()
df["Trust_score"] = df["comment"].progress_apply(classify_trust_score)

# 💾 결과 저장 및 다운로드
output_file = filename.replace(".xlsx", "_trust_scored.xlsx")
df.to_excel(output_file, index=False)
files.download(output_file)

print("✅ 완료: Trust_score (0~1)가 계산되어 저장되었습니다.")

Saving preprocessed_comments_with_length.xlsx to preprocessed_comments_with_length.xlsx


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu
  0%|          | 36/74846 [03:00<120:53:10,  5.82s/it]

2.social engagement

In [ ]:
# 📚 라이브러리 불러오기
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from google.colab import files

# ⬆️ 엑셀 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 엑셀 데이터 불러오기
df = pd.read_excel(filename)

# ✅ 'comment' 열 존재 확인
if 'comment' not in df.columns:
    raise ValueError("❌ 'comment' 열이 엑셀에 없습니다!")

# 🤖 Zero-shot 분류기 로드
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 🎯 social engagement 라벨 정의 (5단계)
social_engagement_labels = [
    ("Very Unrelated", 0.0),
    ("Somewhat Unrelated", 0.25),
    ("Neutral", 0.5),
    ("Somewhat Related", 0.75),
    ("Very Related", 1.0)
]

# 📝 Hypothesis 문장 정의
hypothesis_template = "This comment is {} in terms of the social engagement component of social capital."

# 🧠 social engagement 점수 계산 함수
def classify_social_engagement_score(comment):
    if pd.isna(comment) or not str(comment).strip():
        return 0.0

    labels = [label for label, _ in social_engagement_labels]

    output = classifier(
        comment,
        candidate_labels=labels,
        hypothesis_template=hypothesis_template,
        return_all_scores=True
    )

    # output 검증 및 가중 평균 계산
    if isinstance(output, list):
        result = output
    elif isinstance(output, dict) and "scores" in output:
        result = [
            {'label': label, 'score': score}
            for label, score in zip(output["labels"], output["scores"])
        ]
    else:
        raise ValueError("모델 출력 형식이 예상과 다릅니다")

    weighted = sum(r['score'] * w for r, (_, w) in zip(result, social_engagement_labels))
    return round(weighted, 4)

# 🏃 분석 실행
tqdm.pandas()
df["social_engagement_score"] = df["comment"].progress_apply(classify_social_engagement_score)

# 💾 결과 저장 및 다운로드
output_file = filename.replace(".xlsx", "_social engagement_scored.xlsx")
df.to_excel(output_file, index=False)
files.download(output_file)

print("✅ 완료: social engagement_score (0~1)가 계산되어 저장되었습니다.")

3.Norms of Reciprocity

In [ ]:
# 📚 라이브러리 불러오기
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from google.colab import files

# ⬆️ 엑셀 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 엑셀 데이터 불러오기
df = pd.read_excel(filename)

# ✅ 'comment' 열 존재 확인
if 'comment' not in df.columns:
    raise ValueError("❌ 'comment' 열이 엑셀에 없습니다!")

# 🤖 Zero-shot 분류기 로드
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 🎯 Norms of Reciprocity 라벨 정의 (5단계)
Norms_of_Reciprocity_labels = [
    ("Very Unrelated", 0.0),
    ("Somewhat Unrelated", 0.25),
    ("Neutral", 0.5),
    ("Somewhat Related", 0.75),
    ("Very Related", 1.0)
]

# 📝 Hypothesis 문장 정의
hypothesis_template = "This comment is {} in terms of the Norms of Reciprocity component of social capital."

# 🧠 Norms of Reciprocity 점수 계산 함수
def classify_Norms_of_Reciprocity_score(comment):
    if pd.isna(comment) or not str(comment).strip():
        return 0.0

    labels = [label for label, _ in Norms_of_Reciprocity_labels]

    output = classifier(
        comment,
        candidate_labels=labels,
        hypothesis_template=hypothesis_template,
        return_all_scores=True
    )

    # output 검증 및 가중 평균 계산
    if isinstance(output, list):
        result = output
    elif isinstance(output, dict) and "scores" in output:
        result = [
            {'label': label, 'score': score}
            for label, score in zip(output["labels"], output["scores"])
        ]
    else:
        raise ValueError("모델 출력 형식이 예상과 다릅니다")

    weighted = sum(r['score'] * w for r, (_, w) in zip(result, Norms_of_Reciprocity_labels))
    return round(weighted, 4)

# 🏃 분석 실행
tqdm.pandas()
df["Norms_of_Reciprocity_score"] = df["comment"].progress_apply(classify_Norms_of_Reciprocity_score)

# 💾 결과 저장 및 다운로드
output_file = filename.replace(".xlsx", "_Norms of Reciprocity_scored.xlsx")
df.to_excel(output_file, index=False)
files.download(output_file)

print("✅ 완료: Norms of Reciprocity_score (0~1)가 계산되어 저장되었습니다.")

4. Relational Bonding

In [ ]:
# 📚 라이브러리 불러오기
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from google.colab import files

# ⬆️ 엑셀 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 엑셀 데이터 불러오기
df = pd.read_excel(filename)

# ✅ 'comment' 열 존재 확인
if 'comment' not in df.columns:
    raise ValueError("❌ 'comment' 열이 엑셀에 없습니다!")

# 🤖 Zero-shot 분류기 로드
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 🎯 Relational Bonding 라벨 정의 (5단계)
Relational_Bonding_labels = [
    ("Very Unrelated", 0.0),
    ("Somewhat Unrelated", 0.25),
    ("Neutral", 0.5),
    ("Somewhat Related", 0.75),
    ("Very Related", 1.0)
]

# 📝 Hypothesis 문장 정의
hypothesis_template = "This comment is {} in terms of the Relational_Bonding component of social capital."

# 🧠 Norms of Reciprocity 점수 계산 함수
def classify_Relational_Bonding_score(comment):
    if pd.isna(comment) or not str(comment).strip():
        return 0.0

    labels = [label for label, _ in Relational_Bonding_labels]

    output = classifier(
        comment,
        candidate_labels=labels,
        hypothesis_template=hypothesis_template,
        return_all_scores=True
    )

    # output 검증 및 가중 평균 계산
    if isinstance(output, list):
        result = output
    elif isinstance(output, dict) and "scores" in output:
        result = [
            {'label': label, 'score': score}
            for label, score in zip(output["labels"], output["scores"])
        ]
    else:
        raise ValueError("모델 출력 형식이 예상과 다릅니다")

    weighted = sum(r['score'] * w for r, (_, w) in zip(result, Relational_Bonding_labels))
    return round(weighted, 4)

# 🏃 분석 실행
tqdm.pandas()
df["Relational_Bonding_score"] = df["comment"].progress_apply(classify_Relational_Bonding_score)

# 💾 결과 저장 및 다운로드
output_file = filename.replace(".xlsx", "_Relational_Bonding_scored.xlsx")
df.to_excel(output_file, index=False)
files.download(output_file)

print("✅ 완료: Relational_Bonding_score (0~1)가 계산되어 저장되었습니다.")

5. Affective Connectedness

In [ ]:
# 📚 라이브러리 불러오기
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
from google.colab import files

# ⬆️ 엑셀 파일 업로드
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# 📄 엑셀 데이터 불러오기
df = pd.read_excel(filename)

# ✅ 'comment' 열 존재 확인
if 'comment' not in df.columns:
    raise ValueError("❌ 'comment' 열이 엑셀에 없습니다!")

# 🤖 Zero-shot 분류기 로드
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 🎯 Affective Connectedness 라벨 정의 (5단계)
Affective Connectedness_labels = [
    ("Very Unrelated", 0.0),
    ("Somewhat Unrelated", 0.25),
    ("Neutral", 0.5),
    ("Somewhat Related", 0.75),
    ("Very Related", 1.0)
]

# 📝 Hypothesis 문장 정의
hypothesis_template = "This comment is {} in terms of the Affective Connectedness component of social capital."

# 🧠 Affective Connectedness 점수 계산 함수
def classify_Affective Connectedness_score(comment):
    if pd.isna(comment) or not str(comment).strip():
        return 0.0

    labels = [label for label, _ in Affective Connectedness_labels]

    output = classifier(
        comment,
        candidate_labels=labels,
        hypothesis_template=hypothesis_template,
        return_all_scores=True
    )

    # output 검증 및 가중 평균 계산
    if isinstance(output, list):
        result = output
    elif isinstance(output, dict) and "scores" in output:
        result = [
            {'label': label, 'score': score}
            for label, score in zip(output["labels"], output["scores"])
        ]
    else:
        raise ValueError("모델 출력 형식이 예상과 다릅니다")

    weighted = sum(r['score'] * w for r, (_, w) in zip(result, Affective Connectedness_labels))
    return round(weighted, 4)

# 🏃 분석 실행
tqdm.pandas()
df["Affective Connectedness_score"] = df["comment"].progress_apply(classify_Affective Connectedness_score)

# 💾 결과 저장 및 다운로드
output_file = filename.replace(".xlsx", "_Affective Connectedness_scored.xlsx")
df.to_excel(output_file, index=False)
files.download(output_file)

print("✅ 완료: Affective Connectedness_score (0~1)가 계산되어 저장되었습니다.")